# Data Collection and Preprocessing


Dataset
- Shakespear dataset (nltk built-in)

Task
- Download, Clean and save raw dataset
- Preprocessing - Converting Datasets into tokens and apply padding 
- Save Tokenizer and processed sequences



In [1]:
# Importing required libraries
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
nltk.download('gutenberg')
from nltk.corpus import gutenberg
nltk.download('reuters')
from nltk.corpus import reuters
import pandas as pd
import numpy as np
import pickle

import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Raw Data File Path
raw_data_paths = {
    'hamlet': '../data/raw/hamlet.txt',
    'reuters': '../data/raw/reuters.txt',
}

[nltk_data] Downloading package punkt to /Users/yumna/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package gutenberg to /Users/yumna/nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!
[nltk_data] Downloading package reuters to /Users/yumna/nltk_data...
[nltk_data]   Package reuters is already up-to-date!
/Users/yumna/workspace/next-word-prediction/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


## Data Collection

In [2]:
# Creating a utility function to download nltk datasets and store that in the raw data folder
def download_nltk_dataset(dataset_name):
    """
    Download a dataset from nltk and save it to a file.
    Args:
        dataset_name (str): The name of the dataset to download. ['hamlet', 'reuters']
    """
    if dataset_name == 'hamlet':
        data = gutenberg.raw('shakespeare-hamlet.txt')
    elif dataset_name == 'reuters':
        data = reuters.raw()
    
    with open(raw_data_paths[dataset_name], 'w') as file:
        file.write(data)

    return raw_data_paths[dataset_name]

# Utility function to load the dataset
def load_dataset(dataset_name):
    """
    Load a dataset from a file.
    
    Args:
        dataset_name (str): The name of the dataset to load. ['hamlet', 'reuters']
    """
    with open(raw_data_paths[dataset_name], 'r') as file:
        text = file.read().lower()

    return text



In [4]:
download_nltk_dataset('hamlet')
download_nltk_dataset('reuters')

helmet_text = load_dataset('hamlet')
reuters_text = load_dataset('reuters')

print(len(helmet_text))
print(len(reuters_text))



162881
8846853


In [5]:
# Creating one large corpus
corpus = helmet_text + reuters_text


## Data Preprocessing 

In [6]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts([corpus])

total_words = len(tokenizer.word_index) + 1
total_words

35769

In [7]:
print(tokenizer.word_index)

{'the': 1, 'of': 2, 'to': 3, 'in': 4, 'and': 5, 'a': 6, 'said': 7, 'mln': 8, 'vs': 9, 'for': 10, 'dlrs': 11, 'it': 12, '000': 13, '1': 14, 'pct': 15, 'on': 16, 'lt': 17, 'cts': 18, 'from': 19, 'is': 20, 'that': 21, 'its': 22, 'by': 23, 'year': 24, 'at': 25, 'net': 26, 'be': 27, '2': 28, 'with': 29, 'u': 30, 'will': 31, 's': 32, 'was': 33, 'billion': 34, 'he': 35, 'loss': 36, '3': 37, 'has': 38, 'as': 39, 'would': 40, 'not': 41, '5': 42, 'an': 43, '4': 44, '1986': 45, 'shr': 46, 'company': 47, 'inc': 48, 'but': 49, 'this': 50, 'which': 51, '7': 52, 'bank': 53, '6': 54, 'or': 55, 'are': 56, 'oil': 57, 'corp': 58, 'last': 59, '8': 60, 'share': 61, 'were': 62, 'have': 63, 'trade': 64, 'had': 65, 'one': 66, 'profit': 67, '0': 68, '9': 69, 'about': 70, 'market': 71, 'new': 72, 'two': 73, 'qtr': 74, 'shares': 75, 'stock': 76, 'they': 77, 'also': 78, 'tonnes': 79, '1987': 80, '10': 81, 'revs': 82, 'up': 83, 'sales': 84, 'prices': 85, 'may': 86, 'per': 87, 'been': 88, 'march': 89, 'april': 90, 

In [8]:
# Input sequence 

input_sequences=[]
for line in corpus.split('\n'):
    token_list=tokenizer.texts_to_sequences([line])[0]
    for i in range(1,len(token_list)):
        n_gram_sequence=token_list[:i+1]
        input_sequence=token_list[:i]
        input_sequences.append(n_gram_sequence)


input_sequences[:10]

[[1, 12322],
 [1, 12322, 2],
 [1, 12322, 2, 1597],
 [1, 12322, 2, 1597, 23],
 [1, 12322, 2, 1597, 23, 2020],
 [1, 12322, 2, 1597, 23, 2020, 21939],
 [1, 12322, 2, 1597, 23, 2020, 21939, 21940],
 [16898, 21941],
 [16898, 21941, 21942],
 [16898, 21941, 21942, 21943]]

In [9]:
## Pad Sequences
max_sequence_len=max([len(x) for x in input_sequences])
max_sequence_len

30

In [10]:
input_sequences = np.array(pad_sequences(input_sequences, maxlen = max_sequence_len, padding = 'pre'))
input_sequences

array([[    0,     0,     0, ...,     0,     1, 12322],
       [    0,     0,     0, ...,     1, 12322,     2],
       [    0,     0,     0, ..., 12322,     2,  1597],
       ...,
       [    0,     0,     0, ...,     8,     9,   266],
       [    0,     0,     0, ...,     9,   266,    54],
       [    0,     0,     0, ...,   266,    54,     8]], dtype=int32)

In [11]:
print("Number of input sequences: (like sentences)", len(input_sequences))
print("Input sequences Shape: ", input_sequences.shape)


Number of input sequences: (like sentences) 1322879
Input sequences Shape:  (1322879, 30)


In [12]:
X, y = input_sequences[:,:-1], input_sequences[:,-1]
print("X: ", X)
print("y: ", y)

# Convert y to categorical
y = tf.keras.utils.to_categorical(y, num_classes=total_words)
print("y: ", y)


X:  [[    0     0     0 ...     0     0     1]
 [    0     0     0 ...     0     1 12322]
 [    0     0     0 ...     1 12322     2]
 ...
 [    0     0     0 ...    54     8     9]
 [    0     0     0 ...     8     9   266]
 [    0     0     0 ...     9   266    54]]
y:  [12322     2  1597 ...   266    54     8]
y:  [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [13]:
# Save the tokenizer file and input sequences file as processed data

tokenizerFilePath = '../models/tokenizer.pkl'
inputSequencesFilePath = '../data/processed/processed_sequences.pkl'

# Save the tokenizer file 
with open(tokenizerFilePath, 'wb') as f:
    pickle.dump(tokenizer, f)

# Save the input sequences
with open(inputSequencesFilePath, 'wb') as f:
    pickle.dump(input_sequences, f)
